# We Rate Dogs Twitter:
<p>
Wrangle analyze and visualizations

### Importing Libraries:

In [213]:
# Load libraries
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import json
import os
import requests
import string
import tweepy
from IPython.display import Image
from IPython.core.display import HTML 
%matplotlib inline

### Gather the Data

In [214]:
# Make directory if it doesn't already exist
folder_name = 'image_predictions'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [215]:
# Get data
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

In [216]:
# Create file
with open(os.path.join(folder_name, url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)

In [217]:
predictions = pd.read_csv('image_predictions/image-predictions.tsv', sep='\t')
predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [218]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


### Twitter API

In [219]:
#Variables that contains the user credentials to access Twitter API 
access_token = "21625488-4TKwlwBymT9d8CSkZqhYZ743bXGclWtexGr55SONf"
access_secret = "NAb2boM84TELCAajGu8pl4NDiPxc99n9CGxEhFsBjtiTG"
consumer_key = "Yd1yud9gNqP8uWjcbMQb6Kx4e"
consumer_secret = "RAkiOlRhXQsvrIWLam6iHOHGZo8EUuDvzR3pFJoWf0EQ1vcuHg"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

### Twitter Archive

In [220]:
archive = pd.read_csv('twitter-archive-enhanced.csv')
archive.tweet_id[2000]

672488522314567680

In [221]:
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [222]:
# Get tweet info
tweet = api.get_status(archive.tweet_id[2000], tweet_mode='extended')

In [223]:
# Get json info
info = tweet._json
info

{'created_at': 'Thu Dec 03 18:52:12 +0000 2015',
 'id': 672488522314567680,
 'id_str': '672488522314567680',
 'full_text': 'This is Jackie. She was all ready to go out, but her friends just cancelled on her. 10/10 hang in there Jackie https://t.co/rVfi6CCidK',
 'truncated': False,
 'display_text_range': [0, 134],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 672488519928037376,
    'id_str': '672488519928037376',
    'indices': [111, 134],
    'media_url': 'http://pbs.twimg.com/media/CVUovvHWwAAD-nu.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/CVUovvHWwAAD-nu.jpg',
    'url': 'https://t.co/rVfi6CCidK',
    'display_url': 'pic.twitter.com/rVfi6CCidK',
    'expanded_url': 'https://twitter.com/dog_rates/status/672488522314567680/photo/1',
    'type': 'photo',
    'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'large': {'w': 304, 'h': 411, 'resize': 'fit'},
     'small': {'w': 304, 'h': 411, 'resize': 'f

In [224]:
info['retweet_count']

434

In [225]:
info['favorite_count']

1100

In [226]:
info['user']['followers_count']

8366979

In [227]:
print(datetime.datetime.now().time())

21:03:54.122723


In [228]:
# Make file if it doesn't already exist
file_name = 'tweet_json.txt'
if not os.path.isfile(file_name):
    open(file_name, 'w').close()

In [229]:
tweet_ids = archive.tweet_id

In [230]:

'''tweet_errors = {}
tweet_count = 1
data = []
for tweet_id in tweet_ids:
    try:
        # Print id counter
        print(tweet_count)
        # Collect tweet info
        tweet = api.get_status(tweet_id, tweet_mode='extended')
        info = tweet._json
        #print(info) # debug test
        #break # debug test
        # Append to file
        data.append(info)
        with open(file_name, 'w') as file:
            json.dump(data, file)
        # Print timer info to estimate time until wake-up
        print(datetime.datetime.now().time())
        # Add one to the tweet count for further printing
        tweet_count += 1
        
    except Exception as e:
        # Print exception info and add to tweet_errors dict
        print(str(tweet_id) + ": " + str(e))
        tweet_errors[str(tweet_count - 1) + "_" + str(tweet_id)] = info'''

'tweet_errors = {}\ntweet_count = 1\ndata = []\nfor tweet_id in tweet_ids:\n    try:\n        # Print id counter\n        print(tweet_count)\n        # Collect tweet info\n        tweet = api.get_status(tweet_id, tweet_mode=\'extended\')\n        info = tweet._json\n        #print(info) # debug test\n        #break # debug test\n        # Append to file\n        data.append(info)\n        with open(file_name, \'w\') as file:\n            json.dump(data, file)\n        # Print timer info to estimate time until wake-up\n        print(datetime.datetime.now().time())\n        # Add one to the tweet count for further printing\n        tweet_count += 1\n        \n    except Exception as e:\n        # Print exception info and add to tweet_errors dict\n        print(str(tweet_id) + ": " + str(e))\n        tweet_errors[str(tweet_count - 1) + "_" + str(tweet_id)] = info'

In [231]:
'''data = {}
data['tweets'] = []
tweet_errors = {}
tweet_count = 1
for tweet_id in tweet_ids:
    try:
        # Print id counter
        print(tweet_count)
        # Collect tweet info
        tweet = api.get_status(tweet_id, tweet_mode='extended')
        info = tweet._json
        # Collect specific data
        retweet_count = info['retweet_count']
        favorite_count = info['favorite_count']
        followers_count = info['user']['followers_count']
        # Append to data dict
        data['tweets'].append({
            'tweet_id': tweet_id, 
            'retweet_count': retweet_count, 
            'favorite_count': favorite_count,
            'followers_count': followers_count
        })
        #print(retweet_count, favorite_count, followers_count) # debug test
        #print(data)
        #break # debug test
        # Print timer info to estimate time until wake-up
        print(datetime.datetime.now().time())
        # Add one to the tweet count for further printing
        tweet_count += 1
        
    except Exception as e:
        # Print exception info and add to tweet_errors dict
        print(str(tweet_count) + "_" + str(tweet_id) + ": " + str(e))
        tweet_errors[str(tweet_count) + "_" + str(tweet_id)] = info'''

'data = {}\ndata[\'tweets\'] = []\ntweet_errors = {}\ntweet_count = 1\nfor tweet_id in tweet_ids:\n    try:\n        # Print id counter\n        print(tweet_count)\n        # Collect tweet info\n        tweet = api.get_status(tweet_id, tweet_mode=\'extended\')\n        info = tweet._json\n        # Collect specific data\n        retweet_count = info[\'retweet_count\']\n        favorite_count = info[\'favorite_count\']\n        followers_count = info[\'user\'][\'followers_count\']\n        # Append to data dict\n        data[\'tweets\'].append({\n            \'tweet_id\': tweet_id, \n            \'retweet_count\': retweet_count, \n            \'favorite_count\': favorite_count,\n            \'followers_count\': followers_count\n        })\n        #print(retweet_count, favorite_count, followers_count) # debug test\n        #print(data)\n        #break # debug test\n        # Print timer info to estimate time until wake-up\n        print(datetime.datetime.now().time())\n        # Add one

In [232]:
# Extract data from file
df_list = []
with open('tweet_json.txt') as json_file:
    data = json.load(json_file)
    for tweet in data:
        df_list.append({"tweet_id": tweet["id"],
                        'retweet_count': tweet['retweet_count'], 
                        'favorite_count': tweet['favorite_count'],
                        'followers_count': tweet['user']['followers_count']})

In [233]:
# Create DataFrame from list of dictionaries
api_data = pd.DataFrame(df_list, columns = ['tweet_id', 
                                            'retweet_count', 
                                            'favorite_count', 
                                            'followers_count'])

## Assessing The Data

###  Enhanced Archive Table

In [234]:
#Visually inspect out data.
archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [235]:
#Archive Info Check
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [312]:
#Shape of table
archive.shape

(2356, 17)

In [236]:
# Count number of not 'None' values in'doggo' to 'puppo'  columns.
(archive.loc[:,'doggo':'puppo'] != 'None').sum()

doggo       97
floofer     10
pupper     257
puppo       30
dtype: int64

In [298]:
# Check for missing names.
miss_name = archive[['text', 'name']]
miss_name.query("name.str.startswith('N').values").sample(10)

,text,name
1508,When bae says they can't go out but you see th...,None
352,I couldn't make it to the #WKCDogShow BUT I ha...,None
1150,I'm getting super heckin frustrated with you a...,None
64,@RealKentMurphy 14/10 confirmed,None
1274,"From left to right:\nCletus, Jerome, Alejandro...",None
1552,This pupper just wants to say hello. 11/10 wou...,None
2351,Here we have a 1949 1st generation vulpix. Enj...,None
1938,Guys I'm getting real tired of this. We only r...,None
1826,Hope your Monday isn't too awful. Here's two b...,None
1959,Sun burnt dog here. Quite large. Wants to prom...,None


In [299]:
#Name column value count
archive.name.value_counts()

None       745
a           55
Charlie     12
Oliver      11
Lucy        11
          ... 
Jameson      1
Grady        1
Angel        1
Dewey        1
Sid          1
Name: name, Length: 957, dtype: int64

In [300]:
#Describe raiting numerator column
archive.rating_numerator.describe()

count    2356.000000
mean       13.126486
std        45.876648
min         0.000000
25%        10.000000
50%        11.000000
75%        12.000000
max      1776.000000
Name: rating_numerator, dtype: float64

In [301]:
#Describe rating numerator column
archive.rating_numerator.describe()

count    2356.000000
mean       13.126486
std        45.876648
min         0.000000
25%        10.000000
50%        11.000000
75%        12.000000
max      1776.000000
Name: rating_numerator, dtype: float64

### Predictions Table

In [306]:
#View Predictions Table
predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [307]:
#Predictions info
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [310]:
#shape of predictions
predictions.shape

(2075, 12)

### api_data Table

In [302]:
#View api_data table
api_data.head()

,tweet_id,retweet_count,favorite_count,followers_count
0,892420643555336193,7930,37118,8365519
1,892177421306343426,5891,31934,8365519
2,891815181378084864,3893,24043,8365519
3,891689557279858688,8091,40442,8365519
4,891327558926688256,8774,38669,8365519


In [303]:
#info on table
api_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2333 entries, 0 to 2332
Data columns (total 4 columns):
tweet_id           2333 non-null int64
retweet_count      2333 non-null int64
favorite_count     2333 non-null int64
followers_count    2333 non-null int64
dtypes: int64(4)
memory usage: 73.0 KB


In [311]:
#shape of api_data table
api_data.shape

(2333, 4)

## Findings & Solutions

<h3>Data Exclusion Criteria</h3>
<p>For this project we are expected to exclude.</p>
<ul>
<li>Retweets</li>
<li>Tweets with no images</li>
<li>Reply tweets</li>
</ul>

<p style="text-align: left;"><strong><em>Data Quality Issues</em></strong></p>
<table style="float: left;" width="0">
<tbody>
<tr>
<td width="66">
<p>1.&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;</p>
</td>
<td width="210">
<p>There are 3 separate tables.</p>
</td>
<td width="636">
<p>We fix this by merging our data into one table using &ldquo;twitter_id&rdquo;.</p>
</td>
</tr>
<tr>
<td width="66">
<p>2.&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;</p>
</td>
<td width="210">
<p>Timestamp datatype values are typed as object.</p>
</td>
<td width="636">
<p>We change this to datetime.</p>
</td>
</tr>
<tr>
<td width="66">
<p>3.&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;</p>
</td>
<td width="210">
<p>Re-tweets are included in the dataset these are identified by &ldquo;RT @&rdquo;</p>
<p>&nbsp;</p>
</td>
<td width="636">
<p>We query the text field in our merged data for &ldquo;RT @&rdquo; and drop those columns.</p>
</td>
</tr>
<tr>
<td width="66">
<p>4.&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;</p>
</td>
<td width="210">
<p>Replies are included in the dataset.</p>
<p>&nbsp;</p>
</td>
<td width="636">
<p>Identified in &ldquo;reply_to_status_id&rdquo; Query the table if &ldquo;reply_to_status_id&rdquo;not null then drop rows.</p>
<p>&nbsp;</p>
</td>
</tr>
<tr>
<td width="66">
<p>5.&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;</p>
</td>
<td width="210">
<p>Dog classification name values are &ldquo;NONE&rdquo; should be NaN.</p>
<p>&nbsp;</p>
</td>
<td width="636">
<p>We run a for loop to identify the instance of &ldquo;NONE&rdquo; and we change to NaN.</p>
<p>&nbsp;</p>
</td>
</tr>
<tr>
<td width="66">
<p>6.&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;</p>
</td>
<td width="210">
<p>Redundant columns (doggo, puppo etc).</p>
<p>&nbsp;</p>
</td>
<td width="636">
<p>We Place the values into a new one dimensional column called &ldquo;dog_type&rdquo; and drop redundant columns.</p>
<p>&nbsp;</p>
</td>
</tr>
<tr>
<td width="66">
<p>7.&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;</p>
</td>
<td width="210">
<p>Missing fields for newly created dog type field.</p>
<p>&nbsp;</p>
</td>
<td width="636">
<p>This is not worth fixing, noticing that we may build inaccurate data. We observe that the name could potentially be contained within the text field and mined. However the risks of building false data are too high.</p>
<p>&nbsp;</p>
</td>
</tr>
<tr>
<td width="66">
<p>8.&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;</p>
</td>
<td width="210">
<p>Dog names are missing in the name column.</p>
<p>&nbsp;</p>
</td>
<td width="636">
<p>We fix this by running a for loop and populating our name column based on observable criteria.</p>
<p>&nbsp;</p>
</td>
</tr>
<tr>
<td width="66">
<p>9.&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;</p>
</td>
<td width="210">
<p>Various tweets with images</p>
<p>&nbsp;</p>
</td>
<td width="636">
<p>We query &ldquo;jpg_url&rdquo; in our merged data and exclude the null fields</p>
</td>
</tr>
</tbody>
</table>

<p><strong>Other Notable Quality Issues</strong></p>
<ul>
<li>Erroneous datatypes</li>
<li>Missing info in expanded_urls</li>
<li>Some names are not names just random data</li>
<li>Some rating values are in the 100's when I assume it's max 10</li>
</ul>
<p>&nbsp;</p>

<ol>
<li><em>After gathering the data we merge it into one dataset using outer join based on "Tweet_id" as to not lose data.</em></li>
</ol>
<ul>
<li>First we check our separate dataset shapes.</li>
<li>Then we merge using an outer join based on unique tweet_id.</li>
</ul>

<p>Resources:</p>
<p><a href="http://www.datasciencemadesimple.com/join-merge-data-frames-pandas-python/">Joining Tables</a></p>
<p><a href="https://wesmckinney.com/pages/book.html">Python For Data Analysis -- Wes Mckinney</a></p>

In [239]:
#Api data check
api_data.shape

(2333, 4)

In [240]:
#Archive Shape
archive.shape

(2356, 17)

In [241]:
#Check Predictions
predictions.shape

(2075, 12)

In [242]:
#Merge Api Data and Archive on tweet_id check shape to see any missing data inner join so we don't lose data
twitter_merge = pd.merge(archive, api_data, on='tweet_id', how='outer')
twitter_merge.shape

(2356, 20)

In [243]:
#Merge the final predictions table
final_merge = pd.merge(twitter_merge,predictions, on="tweet_id", how='outer')
final_merge.head()
final_merge.shape

(2356, 31)

In [244]:
final_merge.dtypes

tweet_id                        int64
in_reply_to_status_id         float64
in_reply_to_user_id           float64
timestamp                      object
source                         object
text                           object
retweeted_status_id           float64
retweeted_status_user_id      float64
retweeted_status_timestamp     object
expanded_urls                  object
rating_numerator                int64
rating_denominator              int64
name                           object
doggo                          object
floofer                        object
pupper                         object
puppo                          object
retweet_count                 float64
favorite_count                float64
followers_count               float64
jpg_url                        object
img_num                       float64
p1                             object
p1_conf                       float64
p1_dog                         object
p2                             object
p2_conf     

<p>2. Noticing that timestamp is an object we should change that to a datetime type.</p>
<ul>
<li>Use the pandas pd.to_datetime function we fix this issue.</li>
</ul>

In [245]:
#Fix the timestamp issue
final_merge.timestamp = pd.to_datetime(final_merge.timestamp)

In [246]:
#Check timestamp datatype
final_merge.dtypes.timestamp

datetime64[ns, UTC]

Let's check to see if we have any duplicated data after this
<p>
<li>It would appear we have none.</li>

In [247]:
#Checking our duplicated rows
dups = final_merge[final_merge.duplicated()]
 
print("Duplicate Rows except first occurrence based on all columns are :")
print(dups)

Duplicate Rows except first occurrence based on all columns are :
Empty DataFrame
Columns: [tweet_id, in_reply_to_status_id, in_reply_to_user_id, timestamp, source, text, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, expanded_urls, rating_numerator, rating_denominator, name, doggo, floofer, pupper, puppo, retweet_count, favorite_count, followers_count, jpg_url, img_num, p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog]
Index: []

[0 rows x 31 columns]


In [248]:
final_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 31 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null datetime64[ns, UTC]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
retweet_count      

<p>5. Dog classification name values are &ldquo;NONE&rdquo; should be NaN.</p>
<ul>
<li>We run a for loop to identify the instance of &ldquo;NONE&rdquo; and we change to NaN.</li>
</ul>

In [249]:
dog_types = list(final_merge.iloc[:,13:-14])
dog_types

['doggo', 'floofer', 'pupper', 'puppo']

In [250]:
def find_dog_type(df, dog_type):
    dog_list = []
    for row in df['text']:
        if dog_type in row:
            dog_list.append(dog_type)
        else:
            dog_list.append(np.NaN)
    return dog_list

In [251]:
for dog_type in dog_types:
    final_merge[dog_type] = find_dog_type(final_merge, dog_type)

In [252]:
# Check non-null data counts for columns
final_merge[dog_types].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 4 columns):
doggo      98 non-null object
floofer    4 non-null object
pupper     272 non-null object
puppo      37 non-null object
dtypes: object(4)
memory usage: 92.0+ KB


<p>6. Redundant columns (doggo, puppo etc).</p>
<ul>
<li>We Place the values into a new one dimensional column called &ldquo;dog_type&rdquo; and drop redundant columns.</li>
</ul>

In [253]:
final_merge['dog_type'] = final_merge.puppo.fillna(final_merge.pupper.fillna(final_merge.floofer.fillna(final_merge.doggo)))

In [254]:
final_merge.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace=True)

We notice we have 745 names missing can we find them in the text field?

In [255]:
final_merge.name.value_counts()

None       745
a           55
Charlie     12
Oliver      11
Lucy        11
          ... 
Jameson      1
Grady        1
Angel        1
Dewey        1
Sid          1
Name: name, Length: 957, dtype: int64

<p>8. Some names are missing in the name column.</p>
<ul>
<li>We fix this by running a for loop and populating our name column based on observable criteria.</li>
</ul>
<p>Resources:</p>
<ul>
<li><a href="https://stackoverflow.com/questions/26987222/checking-whitespace-in-a-string-python/26987329">Check for whitespace</a></li>
<li><a href="https://www.geeksforgeeks.org/python-string-split/">Split string</a></li>
<li><a href="https://thispointer.com/python-how-to-check-if-an-item-exists-in-list-search-by-value-or-condition/">Check if list element in string</a></li>
</ul>

In [256]:
def dog_names(df):
    name_list = []
    for row in df['text']:
        # Find first "."
        first_period = row.find(".")
        # If no period is found, assume there is no name
        if first_period == -1:
            name_list.append(np.NaN)
        else:
            # Find word before period
            word_before = row[:first_period].rsplit(' ', 1)[-1]
            # Check if word is capitalized
            if word_before == word_before.title():
                # Add exclusionary criteria - not more than 10 letters, not less than 2 letters, not in other_words, only alphanumeric
                other_words = ["This", "Xbox", "Oh", "Christmas", "Up", "Pupper", "Doggo", "Puppo", "Floofer"]
                if (len(word_before) > 10) or any(word in word_before for word in other_words) or any(c for c in word_before if c not in string.ascii_letters + "'") or (len(word_before) < 2):
                    name_list.append(np.NaN)
                else:
                    name_list.append(word_before)
            else:
                name_list.append(np.NaN)
        
    return name_list

In [257]:
name_list = dog_names(final_merge)
final_merge.name = name_list

In [258]:
# View names and NaNs
final_merge.name.head(10)

0     Phineas
1       Tilly
2      Archie
3       Darla
4    Franklin
5         NaN
6         Jax
7         NaN
8        Zoey
9      Cassie
Name: name, dtype: object

In [259]:
# Check value counts for unexpected names
final_merge.name.value_counts()

Charlie    14
Oliver     12
Cooper     11
Tucker     10
Lucy       10
           ..
Alfy        1
Clarq       1
Joshwa      1
Barclay     1
Biden       1
Name: name, Length: 961, dtype: int64

In [260]:
# Visually compare sample of results
final_merge[['text', 'name']].sample(10)

,text,name
2310,Unfamiliar with this breed. Ears pointy af. Wo...,NaN
184,"THIS IS CHARLIE, MARK. HE DID JUST WANT TO SAY...",NaN
2269,This a Norwegian Pewterschmidt named Tickles. ...,Tickles
246,C'mon guys. Please only send in dogs. We only ...,NaN
1510,This is Bailey. She likes flowers. 12/10 https...,Bailey
1460,"Meet Brian (pronounced ""Kirk""). He's not amuse...",NaN
660,This is Mabel. She's super h*ckin smol. Portab...,Mabel
721,This is Dave. He's currently in a predicament....,Dave
851,This is Tove. She's a Balsamic Poinsetter. Sur...,Tove
496,This is Layla. It is her first Christmas. She ...,Layla


In [261]:
final_merge.dog_type.value_counts()

pupper     272
doggo       86
puppo       37
floofer      4
Name: dog_type, dtype: int64

In [262]:
test_query = final_merge.query("name=='None'")
test_query.count()

tweet_id                      0
in_reply_to_status_id         0
in_reply_to_user_id           0
timestamp                     0
source                        0
text                          0
retweeted_status_id           0
retweeted_status_user_id      0
retweeted_status_timestamp    0
expanded_urls                 0
rating_numerator              0
rating_denominator            0
name                          0
retweet_count                 0
favorite_count                0
followers_count               0
jpg_url                       0
img_num                       0
p1                            0
p1_conf                       0
p1_dog                        0
p2                            0
p2_conf                       0
p2_dog                        0
p3                            0
p3_conf                       0
p3_dog                        0
dog_type                      0
dtype: int64